**Vehicle Price Prediciton**

In [4]:
# prompt: google drive connection

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Step 2: load the dataset
# Upload the CSV to Colab and replace the path below with '/content/vehiclepriceprediction.csv'
file_path = '/content/drive/MyDrive/vehiclepriceprediction.csv'
df = pd.read_csv(file_path)

# Step 3: drop rows with missing target (price) and choose features/target
df = df[df['price'].notnull()]
y = df['price']

# It can be beneficial to drop very high‑cardinality text fields (name and description)
X = df.drop(columns=['price', 'name', 'description'])

# Step 4: define numeric and categorical columns
numeric_features = ['year', 'cylinders', 'mileage', 'doors']
categorical_features = [col for col in X.columns if col not in numeric_features]

# Step 5: build preprocessing for each type of feature
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessing into a single ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Step 6: define and build the regression model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=200, random_state=42))
])

# Step 7: split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

# Step 8: train the model
model.fit(X_train, y_train)

# Step 9: make predictions on the test set
y_pred = model.predict(X_test)

# Step 10: evaluate the model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R² Score: {r2:.4f}")
results = pd.DataFrame({
    'Actual Price': y_test.iloc[:10].values,
    'Predicted Price': y_pred[:10]
})
print(results)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set seaborn style
sns.set(style='whitegrid', palette='muted')

# 1. Distribution of vehicle prices
plt.figure(figsize=(10, 5))
sns.histplot(df['price'], bins=50, kde=True)
plt.title('Distribution of Vehicle Prices')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.show()

# 2. Price vs Year (vehicle depreciation trend)
plt.figure(figsize=(10, 5))
sns.boxplot(data=df, x='year', y='price')
plt.title('Vehicle Price vs Manufacturing Year')
plt.xticks(rotation=45)
plt.ylabel('Price (USD)')
plt.show()

# 3. Average price by make
plt.figure(figsize=(12, 6))
top_makes = df['make'].value_counts().nlargest(10).index
sns.barplot(data=df[df['make'].isin(top_makes)], x='make', y='price', ci=None)
plt.title('Average Price by Top 10 Makes')
plt.ylabel('Average Price (USD)')
plt.xticks(rotation=45)
plt.show()

# 4. Price vs Mileage
plt.figure(figsize=(10, 5))
sns.scatterplot(data=df, x='mileage', y='price', hue='fuel')
plt.title('Price vs Mileage Colored by Fuel Type')
plt.xlabel('Mileage')
plt.ylabel('Price (USD)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

# 5. Correlation heatmap (for numeric features)
plt.figure(figsize=(8, 6))
numeric_df = df[['price', 'year', 'cylinders', 'mileage', 'doors']].dropna()
sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()
